# Project Twitter network

In [5]:
#!pip install bs4
#!pip install requests
#!pip install pandas
#!pip install matplotlib
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
import time 
import matplotlib.pyplot as plt
import matplotlib as mpl


In [6]:
#CLIENT_ID = "pxns5RfNKxD1GG7CQgOSNA"
#SECRET_KEY = "nOYrVLTXMCLMFzf9t53BwHrQJgtpMQ"

In [7]:
#auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

In [8]:
"""
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('<CLIENT_ID>', '<SECRET_TOKEN>')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': '<USERNAME>',
        'password': '<PASSWORD>'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)
"""


'\n# note that CLIENT_ID refers to \'personal use script\' and SECRET_TOKEN to \'token\'\nauth = requests.auth.HTTPBasicAuth(\'<CLIENT_ID>\', \'<SECRET_TOKEN>\')\n\n# here we pass our login method (password), username, and password\ndata = {\'grant_type\': \'password\',\n        \'username\': \'<USERNAME>\',\n        \'password\': \'<PASSWORD>\'}\n\n# setup our header info, which gives reddit a brief description of our app\nheaders = {\'User-Agent\': \'MyBot/0.0.1\'}\n\n# send our request for an OAuth token\nres = requests.post(\'https://www.reddit.com/api/v1/access_token\',\n                    auth=auth, data=data, headers=headers)\n\n# convert response to JSON and pull access_token value\nTOKEN = res.json()[\'access_token\']\n\n# add authorization to our headers dictionary\nheaders = {**headers, **{\'Authorization\': f"bearer {TOKEN}"}}\n\n# while the token is valid (~2 hours) we just add headers=headers to our requests\nrequests.get(\'https://oauth.reddit.com/api/v1/me\', headers=h

In [9]:
import openai
import requests

In [25]:
openai.api_key = ""

In [35]:
def send_message_to_chatgpt(prompt):
    model_engine = "text-davinci-003" #davinici codex -> code, curie -> mindre, babbage -> smaller endnu, ada -> 
    
    
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=250,
        n=1,
        stop=None,
        temperature=0.7,
    )

    return response.choices[0].text.strip()


message = "What is a mountain range?"
response = send_message_to_chatgpt(message)
print(response)

As an AI language model, I don't have any information about a "ducking mountain range." However, if you meant "duckling mountain range," there is no such thing as a duckling mountain range. It is possible that you may have misspelled or misheard the term. Please provide more context or information so I can assist you better.


In [40]:
def prompting(prompt):
  completion = openai.ChatCompletion.create( # 1. Change the function Completion to ChatCompletion
  model = 'gpt-3.5-turbo',
  messages = [ # 2. Change the prompt parameter to the messages parameter
    {'role': 'user', 'content':prompt}
  ],
  temperature = 0  
  )

  return completion

prompt = "write python code to put ALL members of the paypall mafia in a dataframe, only write the code not anything else, its important that all 19 are included"
answer = prompting(prompt)['choices'][0]['message']['content']

In [41]:
print(answer)

import pandas as pd

members = {'Peter Thiel': 'Co-founder',
           'Elon Musk': 'Co-founder',
           'Max Levchin': 'Co-founder',
           'Reid Hoffman': 'Early employee',
           'Roelof Botha': 'Early employee',
           'David Sacks': 'COO',
           'Chad Hurley': 'Co-founder of YouTube',
           'Steve Chen': 'Co-founder of YouTube',
           'Jeremy Stoppelman': 'Co-founder of Yelp',
           'Russel Simmons': 'Co-founder of Yelp',
           'Keith Rabois': 'COO of Square',
           'Jack Selby': 'Early employee of PayPal and investor',
           'Yishan Wong': 'Director of Engineering',
           'Jawed Karim': 'Co-founder of YouTube',
           'Premal Shah': 'President of Kiva.org',
           'Luke Nosek': 'Co-founder',
           'Ken Howery': 'Co-founder',
           'Yu Pan': 'Co-founder',
           'Dave McClure': 'Early employee and investor'}

df = pd.DataFrame(list(members.items()), columns=['Name', 'Position'])
